In [1]:
import os

In [2]:
%pwd

'd:\\1Semester 6\\ML\\Project\\CO544-MLOPs\\research'

In [3]:
!cd ..

In [4]:
%pwd

'd:\\1Semester 6\\ML\\Project\\CO544-MLOPs\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'd:\\1Semester 6\\ML\\Project\\CO544-MLOPs'

In [57]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [58]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories




In [60]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [61]:
import os
import urllib.request as request
import requests
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [64]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading file from {self.config.source_URL}")
            response = requests.get(self.config.source_URL, stream=True)
            if response.status_code == 200:
                with open(self.config.local_data_file, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                logger.info(f"File downloaded successfully: {self.config.local_data_file}")
            else:
                logger.error(f"Failed to download file: status code {response.status_code}")
                response.raise_for_status()
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [66]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-23 14:40:11,263: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-23 14:40:11,265: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-23 14:40:11,266: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-23 14:40:11,267: INFO: common: created directory at: artifacts]
[2024-06-23 14:40:11,268: INFO: common: created directory at: artifacts/data_ingestion]
